In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/stan'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1]).reshape(-1,1)
b_value = np.array([0.0,0.25,0.5,1.0]).reshape(-1,1)


LR_tune, B_value = np.meshgrid(lr_tune,b_value)

LR_tune = LR_tune.flatten('F').reshape(-1,1)
B_value = B_value.flatten('F').reshape(-1,1)


lrb_tune = np.hstack((LR_tune,B_value))

In [6]:
def true_1D_2(x): #True function for 1D_1 dy/dx = cos(0.01*x) BC1: y(0)=0; x \in [-100,100]
    y = 100*np.sin(0.01*x)
    return y
    

In [7]:
loss_thresh = 0.005

x = np.linspace(-600,600,5000).reshape(-1,1)
ysol = true_1D_2(x)

bc1_x = np.array(0).reshape(-1,1) 
bc1_y = np.array(0).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)

 
x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)

y_true = true_1D_2(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

#torch.autograd.set_detect_anomaly(True)

In [8]:
def colloc_pts(N_f,seed):

  #Collocation Points
  # Latin Hypercube sampling for collocation points 
  # N_f sets of tuples(x,y)
  x01 = np.array([[0.0, 1.0]])
  sampling = LHS(xlimits=x01,random_state =seed)

  x_coll_train = lb + (ub-lb)*sampling(N_f)
  x_coll_train = np.vstack((x_coll_train, bc1_x)) # append training points to collocation points 

  return x_coll_train

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,beta_init):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
        
        self.beta = Parameter(beta_init*torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True
        
        self.iter = 0
        
              
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z) + self.beta[:,i]*z*self.activation(z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        f = dy_dx - torch.cos(0.01*g)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + 100*loss_f
        
        return loss_val
     
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred

    def test_loss(self):
        y_pred = self.test()
        
       
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
      
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re 

In [10]:
def train_step(seed):
    x_coll_np_array = colloc_pts(N_f,seed*123)
    x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_coll_train,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        
        return loss

    optimizer.step(closure)

In [11]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [12]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    nan_flag = 0
    
    x_coll = torch.from_numpy(colloc_pts(N_f,123)).float().to(device)
    f_hat = torch.zeros(x_coll.shape[0],1).to(device)

    for i in range(max_iter):
        train_step(i)
    
        loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
        
        if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break

    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))
    
    return nan_flag

In [13]:
nan_tune = []

for tune_reps in range(20):
    label = "1D_FODE_stan_tune"+str(tune_reps)  
    max_reps = 10
    max_iter = 100

    train_loss_full = []
    test_mse_full = []
    test_re_full = []
    beta_full = []
    elapsed_time= np.zeros((max_reps,1))
    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))

    beta_init = lrb_tune[tune_reps,1]


    for reps in range(max_reps):
        print(label)
        print(reps)
        train_loss = []
        test_mse_loss = []
        test_re_loss = []   
        beta_val = []
      
        torch.manual_seed(reps*36)
        N_f = 10000 #Total number of collocation points
    
        layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
        PINN = Sequentialmodel(layers,beta_init)
    
        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())
      
        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lrb_tune[tune_reps,0], 
                                max_iter = 10, 
                                max_eval = 15, 
                                tolerance_grad = 1e-6, 
                                tolerance_change = 1e-6, 
                                history_size = 100, 
                                line_search_fn = 'strong_wolfe')
    

      
        nan_flag = train_model(max_iter,reps)

      
        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)
        beta_full.append(beta_val)
        
        if(nan_flag == 1):
            nan_tune.append(tune_reps)
            break
      
      
        print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full, "label": label}
    savemat(label+'.mat', mdic)

1D_FODE_stan_tune0
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 47.768345 Test MSE 5220.110007971045 Test RE 0.9997536361627732
1 Train Loss 47.76822 Test MSE 5220.011495850385 Test RE 0.9997442026148837
2 Train Loss 47.76788 Test MSE 5219.14438658653 Test RE 0.9996611641518973
3 Train Loss 47.75574 Test MSE 5175.762605508289 Te

97 Train Loss 3.239898 Test MSE 2300.4582332144323 Test RE 0.663682461364546
98 Train Loss 3.198442 Test MSE 2315.3405441082014 Test RE 0.6658257747196087
99 Train Loss 3.1379573 Test MSE 2260.3794184347325 Test RE 0.6578756873799827
Training time: 88.45
Training time: 88.45
1D_FODE_stan_tune0
1
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
1
0 T

93 Train Loss 1.0160174 Test MSE 765.59149349745 Test RE 0.3828703315341396
94 Train Loss 0.9754929 Test MSE 696.5401575009261 Test RE 0.36519619552352206
95 Train Loss 0.94207096 Test MSE 682.3656248608027 Test RE 0.36146124068009017
96 Train Loss 0.9238634 Test MSE 642.0095791698 Test RE 0.3506096931784588
97 Train Loss 0.91513926 Test MSE 651.6890130351054 Test RE 0.3532428371769344
98 Train Loss 0.9096583 Test MSE 657.3802846344738 Test RE 0.354781938409144
99 Train Loss 0.9075377 Test MSE 658.5279872945364 Test RE 0.3550915055013958
Training time: 77.56
Training time: 77.56
1D_FODE_stan_tune0
2
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)


89 Train Loss 19.024237 Test MSE 3795.502312594433 Test RE 0.8524870587417748
90 Train Loss 18.962437 Test MSE 3734.7782248954436 Test RE 0.8456401093489697
91 Train Loss 18.908827 Test MSE 3767.779893262416 Test RE 0.849368062352906
92 Train Loss 18.842035 Test MSE 3917.596531008069 Test RE 0.8660899874355499
93 Train Loss 18.598522 Test MSE 4181.40576019417 Test RE 0.8947759886421196
94 Train Loss 18.186924 Test MSE 4192.913124149347 Test RE 0.8960063689492018
95 Train Loss 17.528254 Test MSE 4205.68615584527 Test RE 0.8973701003862983
96 Train Loss 17.241331 Test MSE 4194.391058518622 Test RE 0.8961642689456865
97 Train Loss 17.18789 Test MSE 4086.90488841679 Test RE 0.884607118977894
98 Train Loss 17.18293 Test MSE 4075.741943708082 Test RE 0.8833981879093822
99 Train Loss 17.126959 Test MSE 3998.0377569832 Test RE 0.8749366526757533
Training time: 64.62
Training time: 64.62
1D_FODE_stan_tune0
3
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): Modul

85 Train Loss 8.479495 Test MSE 3743.4634584005385 Test RE 0.8466228071648896
86 Train Loss 8.396978 Test MSE 3883.956384372621 Test RE 0.86236344108649
87 Train Loss 8.331607 Test MSE 4085.426361710988 Test RE 0.8844470915734939
88 Train Loss 8.2182255 Test MSE 4120.615261946398 Test RE 0.8882479176876871
89 Train Loss 8.166362 Test MSE 3855.604513242749 Test RE 0.8592101616355908
90 Train Loss 8.114278 Test MSE 3488.280730993649 Test RE 0.8172574329188156
91 Train Loss 8.006024 Test MSE 3226.3464055547533 Test RE 0.7859748727469563
92 Train Loss 7.9506116 Test MSE 3320.493978536096 Test RE 0.7973601247254579
93 Train Loss 7.8922796 Test MSE 3237.5704608035767 Test RE 0.7873408398655332
94 Train Loss 7.8291974 Test MSE 3174.779535735396 Test RE 0.7796684326628756
95 Train Loss 7.7380123 Test MSE 3110.213748557403 Test RE 0.7716996141231705
96 Train Loss 7.7133794 Test MSE 3039.6598099773364 Test RE 0.7628965577798975
97 Train Loss 7.707453 Test MSE 2980.7386644402213 Test RE 0.7554663

...

85 Train Loss 0.010891705 Test MSE 0.0772390726476493 Test RE 0.0038456671289980944
86 Train Loss 0.010806353 Test MSE 0.10359020295265352 Test RE 0.004453613314300539
87 Train Loss 0.010715118 Test MSE 0.11277792755150942 Test RE 0.004646920252587972
88 Train Loss 0.010529275 Test MSE 0.14754347789384142 Test RE 0.005315121209327892
89 Train Loss 0.010529275 Test MSE 0.14754376399723237 Test RE 0.005315126362634178
90 Train Loss 0.010529271 Test MSE 0.14754375129714553 Test RE 0.005315126133879788
91 Train Loss 0.010529271 Test MSE 0.14754375129714553 Test RE 0.005315126133879788
92 Train Loss 0.010529271 Test MSE 0.14754366347009693 Test RE 0.0053151245519357645
93 Train Loss 0.010529271 Test MSE 0.14754366347009693 Test RE 0.0053151245519357645
94 Train Loss 0.010529271 Test MSE 0.14754366347009693 Test RE 0.0053151245519357645
95 Train Loss 0.010529266 Test MSE 0.14754380815878035 Test RE 0.005315127158073388
96 Train Loss 0.010529245 Test MSE 0.14754375515343565 Test RE 0.00531512

77 Train Loss 0.007290986 Test MSE 0.06524860139797502 Test RE 0.003534587751812481
78 Train Loss 0.0067742704 Test MSE 0.050569363790047034 Test RE 0.003111694194470287
79 Train Loss 0.006451408 Test MSE 0.03432212080814125 Test RE 0.002563540733005965
80 Train Loss 0.0062877648 Test MSE 0.01888597900493636 Test RE 0.0019016164077061274
81 Train Loss 0.006088767 Test MSE 0.03188641564552495 Test RE 0.0024709048175200914
82 Train Loss 0.0058369283 Test MSE 0.04020206281845908 Test RE 0.0027744527537980714
83 Train Loss 0.0052795275 Test MSE 0.10260562652523658 Test RE 0.004432398027857751
84 Train Loss 0.005003877 Test MSE 0.15684380249425223 Test RE 0.005480079337961184
85 Train Loss 0.0048832735 Test MSE 0.15448940648786763 Test RE 0.005438792838711337
86 Train Loss 0.0048323707 Test MSE 0.15781932830102052 Test RE 0.005497095222174855
87 Train Loss 0.004829509 Test MSE 0.1676978851551323 Test RE 0.005666526936083903
88 Train Loss 0.0047433134 Test MSE 0.16282336854744622 Test RE 0.0

69 Train Loss 0.092939265 Test MSE 0.3980109805164545 Test RE 0.008729727584871689
70 Train Loss 0.0897363 Test MSE 0.5256922829942556 Test RE 0.010032726323402118
71 Train Loss 0.08773367 Test MSE 0.6285121862712277 Test RE 0.010970085590166618
72 Train Loss 0.08705415 Test MSE 0.9568140214641732 Test RE 0.013535270209486975
73 Train Loss 0.08611484 Test MSE 0.5537500910637212 Test RE 0.010296984760879922
74 Train Loss 0.08477502 Test MSE 0.28157856986355456 Test RE 0.007342652312541896
75 Train Loss 0.08105252 Test MSE 0.19052582819700326 Test RE 0.006039904804895002
76 Train Loss 0.07659616 Test MSE 0.4199336933250187 Test RE 0.008966924755809525
77 Train Loss 0.0740984 Test MSE 0.48483693787584486 Test RE 0.009634984370752095
78 Train Loss 0.071623474 Test MSE 0.3657699716102603 Test RE 0.00836868437945786
79 Train Loss 0.068149194 Test MSE 0.7528828496425632 Test RE 0.012006512207878978
80 Train Loss 0.06649857 Test MSE 1.367651496762046 Test RE 0.016182323811136212
81 Train Loss 

61 Train Loss 0.0062598553 Test MSE 0.007719940120279882 Test RE 0.0012157943772572693
62 Train Loss 0.006231031 Test MSE 0.008024955259351778 Test RE 0.0012395797547720415
63 Train Loss 0.00605628 Test MSE 0.00880958658672269 Test RE 0.0012987660540462352
64 Train Loss 0.005617781 Test MSE 0.008456591335669216 Test RE 0.0012724796234674964
65 Train Loss 0.0052685277 Test MSE 0.007914152077563278 Test RE 0.0012309923679887846
66 Train Loss 0.0052152476 Test MSE 0.006658621692533379 Test RE 0.0011291336111407795
67 Train Loss 0.0051654256 Test MSE 0.006451009773026422 Test RE 0.001111391355791476
68 Train Loss 0.005027958 Test MSE 0.027367832327490238 Test RE 0.002289145327214395
69 Train Loss 0.0046417215 Test MSE 0.046831715248845 Test RE 0.002994492273962412
70 Train Loss 0.004457579 Test MSE 0.025877468203032078 Test RE 0.0022259430890782376
71 Train Loss 0.0043856762 Test MSE 0.017590831371768718 Test RE 0.001835254710290649
72 Train Loss 0.0042864853 Test MSE 0.014054011717318281 

52 Train Loss 0.018337548 Test MSE 2.0593831531139353 Test RE 0.01985737066137688
53 Train Loss 0.01735671 Test MSE 1.7422025577102294 Test RE 0.018264277127663864
54 Train Loss 0.016924601 Test MSE 1.7428555141259026 Test RE 0.018267699421953217
55 Train Loss 0.016509442 Test MSE 1.8776609687827752 Test RE 0.01896102251796675
56 Train Loss 0.016179517 Test MSE 1.635552991368758 Test RE 0.017696422338769485
57 Train Loss 0.01582168 Test MSE 1.4615618668920587 Test RE 0.016728683571229367
58 Train Loss 0.015562368 Test MSE 1.4624593655383529 Test RE 0.016733819059164046
59 Train Loss 0.015374492 Test MSE 1.3853390416187288 Test RE 0.016286628925271995
60 Train Loss 0.015168399 Test MSE 1.252028920394986 Test RE 0.015483186499314973
61 Train Loss 0.014948189 Test MSE 0.9856044281247665 Test RE 0.01373739821755721
62 Train Loss 0.013635831 Test MSE 0.9835565608009723 Test RE 0.013723119163746699
63 Train Loss 0.013195248 Test MSE 0.9991010295918928 Test RE 0.013831136516769893
64 Train Lo

In [14]:
label

'1D_FODE_stan_tune19'

In [15]:
import scipy.io as sio

In [19]:
for tune_reps in range(20):
    label = "1D_FODE_stan_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps, " ",np.mean(re[:,-1])<0.0021)

0   False
1   False
2   False
3   False
4   False
5   False
6   False
7   False
8   False
9   False
10   False
11   False
12   False
13   False
14   [[False False False False]]
15   True
16   [[False False False]]
17   False
18   [[False False False False]]
19   [[False False False]]


In [20]:
lrb_tune[15]

array([0.5, 1. ])